In [84]:
import uproot as up
import awkward as ak
import coffea
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema, TreeMakerSchema
from coffea.nanoevents.methods import candidate
from coffea import lookup_tools
from coffea.lookup_tools import extractor
from coffea.jetmet_tools import FactorizedJetCorrector, JetCorrectionUncertainty
from coffea.jetmet_tools import JECStack, CorrectedJetsFactory, CorrectedMETFactory
from coffea.btag_tools.btagscalefactor import BTagScaleFactor

ak.behavior.update(candidate.behavior)
import numpy as np

In [93]:
file_path = "/data/pubfs/xiaoj/download/datasets/nanov9/SingleMuon/Run2018A-UL2018_MiniAODv2_NanoAODv9-v2/NANOAOD/1F6BA314-D63C-FD45-9DFF-A5E4C475F4A3.root"
file_path = "/data/pubfs/xiaoj/download/datasets/nanov9/VBFHToTauTau_M125_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM/AA97740E-6A61-4C48-B260-22E3DDEB4C0E.root"
# events = up.open(f"{file_path}:Events")
events = NanoEventsFactory.from_root(file_path, schemaclass=NanoAODSchema).events()

In [94]:
# taus = events.Tau
# boost_taus = events.boostedTau
coffea.__version__

'0.7.11'

In [95]:
# good pv
sel_pv = events.PV.npvsGood > 0
events = events.mask[sel_pv]

In [96]:
# muons =events['Muon_pt']
# muons

In [97]:
# muons
muons = events.Muon
sel_mu_1 = (muons.mediumId) & (muons.pt > 15)  & (abs(muons.eta) < 2.4)
sel_mu_2 = (abs(muons.dxy) < 0.2) & (abs(muons.dz) < 0.5)
sel_mu_3 = (muons.pfIsoId >= 2) # loose relPFIso

good_muon_mask = (sel_mu_1) & (sel_mu_2) & (sel_mu_3) 
# Rochester correction

# Dress the muons by FSR Photons
sel_good_muon = (ak.sum(good_muon_mask,axis=1) > 1)
muons = muons.mask[sel_good_muon]
good_muons = muons[good_muon_mask]
events = events.mask[sel_good_muon]



In [98]:
# https://gitlab.cern.ch/akhukhun/roccor
# https://github.com/CoffeaTeam/coffea/blob/master/coffea/lookup_tools/rochester_lookup.py
# https://github.com/TopEFT/topcoffea/blob/master/topcoffea/modules/corrections.py#L359
def apply_rochester_correction(mu, is_mc=True, year='2018'):
    if year=='2016': rochester_data = lookup_tools.txt_converters.convert_rochester_file("data/MuonScale/RoccoR2016.txt", loaduncs=True)
    elif year=='2017': rochester_data = lookup_tools.txt_converters.convert_rochester_file("data/MuonScale/RoccoR2017.txt", loaduncs=True)
    elif year=='2018': rochester_data = lookup_tools.txt_converters.convert_rochester_file("/data/pubfs/xiaoj/test/roc/RoccoR/RoccoR2018.txt", loaduncs=True)
    rochester = lookup_tools.rochester_lookup.rochester_lookup(rochester_data)
    if is_mc:
        hasgen = ~np.isnan(ak.fill_none(mu.matched_gen.pt, np.nan))
        mc_rand = np.random.rand(*ak.to_numpy(ak.flatten(mu.pt)).shape)
        mc_rand = ak.unflatten(mc_rand, ak.num(mu.pt, axis=1))
        corrections = np.array(ak.flatten(ak.ones_like(mu.pt)))
        errors = np.array(ak.flatten(ak.ones_like(mu.pt)))
        
        mc_kspread = rochester.kSpreadMC(mu.charge[hasgen],mu.pt[hasgen],mu.eta[hasgen],mu.phi[hasgen],mu.matched_gen.pt[hasgen])
        mc_ksmear = rochester.kSmearMC(mu.charge[~hasgen],mu.pt[~hasgen],mu.eta[~hasgen],mu.phi[~hasgen],mu.nTrackerLayers[~hasgen],mc_rand[~hasgen])
        errspread = rochester.kSpreadMCerror(mu.charge[hasgen],mu.pt[hasgen],mu.eta[hasgen],mu.phi[hasgen],mu.matched_gen.pt[hasgen])
        errsmear = rochester.kSmearMCerror(mu.charge[~hasgen],mu.pt[~hasgen],mu.eta[~hasgen],mu.phi[~hasgen],mu.nTrackerLayers[~hasgen],mc_rand[~hasgen])
        hasgen_flat = np.array(ak.flatten(hasgen))
        corrections[hasgen_flat] = np.array(ak.flatten(mc_kspread))
        corrections[~hasgen_flat] = np.array(ak.flatten(mc_ksmear))
        errors[hasgen_flat] = np.array(ak.flatten(errspread))
        errors[~hasgen_flat] = np.array(ak.flatten(errsmear))
        corrections = ak.unflatten(corrections, ak.num(mu.pt, axis=1))
        errors = ak.unflatten(errors, ak.num(mu.pt, axis=1))
    else:
        corrections = rochester.kScaleDT(mu.charge, mu.pt, mu.eta, mu.phi)
        errors = rochester.kScaleDTerror(mu.charge, mu.pt, mu.eta, mu.phi)
    
    pt_nom = mu.pt * corrections
    pt_err = mu.pt * errors
    # tmp = np.array(ak.flatten(pt_nom + pt_err))
    # tmp[tmp < 0] = 0
    # pt_up = ak.unflatten(tmp, ak.num(pt_nom, axis=1))
    # tmp = np.array(ak.flatten(pt_nom - pt_err))
    # tmp[tmp < 0] = 0
    # pt_down = ak.unflatten(tmp, ak.num(pt_nom, axis=1))
    return pt_nom, pt_nom + pt_err, pt_nom - pt_err

In [33]:
good_muons['newpt'] , good_muons['newpt_up'] , good_muons['newpt_down'] = apply_rochester_correction(good_muons)

In [91]:
# final_mask = ak.fill_none(events.run!=None, False)
# good_muons=good_muons[final_mask]
# events = events[final_mask]


In [92]:
# with up.recreate("new_test.root", compression=None) as fout:
#     fout['Events'] = {
#         'Muon': good_muons,
#     }

# hah = up.lazy("new_test.root:Events")
# events = NanoEventsFactory.from_root("new_test.root", schemaclass=BaseSchema).events()

In [99]:
eles = events.Electron
# electrons
good_ele_mask = (eles.pt > 20) & (abs(eles.eta + eles.deltaEtaSC) < 2.5) & (eles.mvaFall17V2Iso_WP90)

sel_no_ele = (ak.sum(good_ele_mask,axis=1) < 1)

events = events.mask[sel_no_ele]

In [100]:
def is_clean(obj_A, obj_B, drmin=0.4):
   ## Method 1
   # pair_obj = ak.cartesian([obj_A, obj_B],nested=True)
   # obj1, obj2 = ak.unzip(pair_obj)
   # dr_jm = obj1.delta_r(obj2)
   # min_dr_jm = ak.min(dr_jm,axis=2)
   # mask = min_dr_jm > drmin
   
   ## Method 2
   objB_near, objB_dr = obj_A.nearest(obj_B, return_metric=True)
   mask = ak.fill_none(objB_dr > drmin, True) # I guess to use True is because if there are no objB, all the objA are clean
   return (mask)


In [101]:
# jets
jets = events.Jet
# jet cleaned w.r.t. muons
clean_jet_mask = is_clean(jets, good_muons, 0.4)
# clean jets
sel_clean_jet = (ak.sum(clean_jet_mask,axis=1) > 0)
jets = jets.mask[sel_clean_jet]
clean_jets = jets[clean_jet_mask]
events = events.mask[sel_clean_jet]
# sel_cleanj_1 = (clean_jets.pt > 25) & (abs(clean_jets.eta) < 4.7) & (clean_jets.isTight)

# bjets 
# sel_bjet = (sel_jet) & 

In [102]:
total_sel = ak.fill_none(events.run!=None, False)
events = events[total_sel]
good_muons = good_muons[total_sel]
clean_jets = clean_jets[total_sel]


In [162]:
trigger = {}
for idx, ihlt in enumerate(['IsoMu24']):
    trigger[ihlt] = events.HLT[ihlt]
jet_dict = {}
for ibr in ['area', 'btagCSVV2', 'btagDeepB']:
    # cross indexing should be romoved, e.g., muonIdxG
    if (not 'Idx' in ibr) and (not ibr.endswith('G')):
        jet_dict[ibr] = ak.Array(clean_jets[ibr].to_list())

In [163]:
jet_dict

{'area': <Array [[0.469, 0.509], ... 0.479, 0.519]] type='824 * var * float64'>,
 'btagCSVV2': <Array [[0.157, -10], ... 0.187, 0.153, -10]] type='824 * var * float64'>,
 'btagDeepB': <Array [[0.077, -1], ... 0.0511, -1, -1]] type='824 * var * float64'>}

In [158]:
good_muons.pt.dtype

AttributeError: no field named 'dtype'

(https://github.com/scikit-hep/awkward-1.0/blob/1.7.0/src/awkward/highlevel.py#L1131)

In [167]:
with up.recreate("new_test.root", compression=None) as fout:
    fout['Events'] = {
        'Jet': jet_dict,
        # 'basic': {
        #     'run':events.run,
        #     'luminosityBlock': events.luminosityBlock,
        #     'event': events.event,
        # },
        # 'Muon': good_muons,
        # 'MET': events.MET,
        # 'PuppiMET': events.PuppiMET,
        # 'HLT': trigger,
    }

AttributeError: no field named 'dtype'

(https://github.com/scikit-hep/awkward-1.0/blob/1.7.0/src/awkward/highlevel.py#L1131)

In [124]:
clean_jets.fields

['area',
 'btagCSVV2',
 'btagDeepB',
 'btagDeepCvB',
 'btagDeepCvL',
 'btagDeepFlavB',
 'btagDeepFlavCvB',
 'btagDeepFlavCvL',
 'btagDeepFlavQG',
 'chEmEF',
 'chFPV0EF',
 'chHEF',
 'eta',
 'hfsigmaEtaEta',
 'hfsigmaPhiPhi',
 'mass',
 'muEF',
 'muonSubtrFactor',
 'neEmEF',
 'neHEF',
 'phi',
 'pt',
 'puIdDisc',
 'qgl',
 'rawFactor',
 'bRegCorr',
 'bRegRes',
 'cRegCorr',
 'cRegRes',
 'electronIdx1',
 'electronIdx2',
 'hfadjacentEtaStripsSize',
 'hfcentralEtaStripSize',
 'jetId',
 'muonIdx1',
 'muonIdx2',
 'nElectrons',
 'nMuons',
 'puId',
 'nConstituents',
 'genJetIdx',
 'hadronFlavour',
 'partonFlavour',
 'cleanmask',
 'electronIdx1G',
 'electronIdx2G',
 'genJetIdxG',
 'muonIdx1G',
 'muonIdx2G',
 'muonIdxG',
 'electronIdxG']

In [61]:
### JES JER
def apply_jet_corrections(year, corr_type="jet"):
    extract = extractor()
    extract.add_weight_sets(
        [
            "* * ./jes/Autumn18_V19_MC_L1FastJet_AK4PFchs.jec.txt",
            "* * ./jes/Autumn18_V19_MC_L2Relative_AK4PFchs.jec.txt",
            "* * ./jes/Autumn18_V19_MC_L3Absolute_AK4PFchs.jec.txt",
            "* * ./jes/Autumn18_V19_MC_L2L3Residual_AK4PFchs.jec.txt",
            "* * ./jes/Regrouped_Autumn18_V19_MC_UncertaintySources_AK4PFchs.junc.txt",
            "* * ./jerUL/Summer19UL18_JRV2_MC_SF_AK4PFchs.jersf.txt",
            "* * ./jerUL/Summer19UL18_JRV2_MC_PtResolution_AK4PFchs.jr.txt",
        ]
    )

    extract.finalize()
    evaluator = extract.make_evaluator()

    jec_names = dir(evaluator)
    print(jec_names)
    jec_inputs = {name: evaluator[name] for name in jec_names}
    jec_stack = JECStack(jec_inputs)
    name_map = jec_stack.blank_name_map
    name_map['JetPt'] = 'pt'
    name_map['JetMass'] = 'mass'
    name_map['JetEta'] = 'eta'
    name_map['JetPhi'] = 'phi'
    name_map['JetA'] = 'area'
    name_map['ptGenJet'] = 'pt_gen'
    name_map['ptRaw'] = 'pt_raw'
    name_map['massRaw'] = 'mass_raw'
    name_map['Rho'] = 'rho'
    name_map['METpt'] = 'pt'
    name_map['METphi'] = 'phi'
    name_map['UnClusteredEnergyDeltaX'] = 'MetUnclustEnUpDeltaX'
    name_map['UnClusteredEnergyDeltaY'] = 'MetUnclustEnUpDeltaY'
    if corr_type=='met': return CorrectedMETFactory(name_map)
    return CorrectedJetsFactory(name_map, jec_stack)

    return

In [72]:
is_data = False
met = events.MET

if not is_data:
    clean_jets["pt_raw"] = (1 - clean_jets.rawFactor)*clean_jets.pt
    clean_jets["mass_raw"] = (1 - clean_jets.rawFactor)*clean_jets.mass
    clean_jets["pt_gen"] =ak.values_astype(ak.fill_none(clean_jets.matched_gen.pt, 0), np.float32)
    clean_jets["rho"] = ak.broadcast_arrays(events.fixedGridRhoFastjetAll, clean_jets.pt)[0]
    events_cache = events.caches[0]
    new_clean_jets = apply_jet_corrections(2018, corr_type='jets').build(clean_jets, lazy_cache=events_cache)
    # new_met = apply_jet_corrections(2018, corr_type='met').build(met, new_clean_jets, lazy_cache=events_cache)
    # # SYSTEMATICS
    # cleanedJets=ApplyJetSystematics(cleanedJets,syst_var)
    # met=ApplyJetCorrections(year, corr_type='met').build(met_raw, cleanedJets, lazy_cache=events_cache)
    
    # clean_jets['JER_up'] = new_clean_jets.JER.up

['Autumn18_V19_MC_L1FastJet_AK4PFchs', 'Autumn18_V19_MC_L2L3Residual_AK4PFchs', 'Autumn18_V19_MC_L2Relative_AK4PFchs', 'Autumn18_V19_MC_L3Absolute_AK4PFchs', 'Regrouped_Autumn18_V19_MC_UncertaintySources_AK4PFchs_Absolute', 'Regrouped_Autumn18_V19_MC_UncertaintySources_AK4PFchs_Absolute_2018', 'Regrouped_Autumn18_V19_MC_UncertaintySources_AK4PFchs_BBEC1', 'Regrouped_Autumn18_V19_MC_UncertaintySources_AK4PFchs_BBEC1_2018', 'Regrouped_Autumn18_V19_MC_UncertaintySources_AK4PFchs_EC2', 'Regrouped_Autumn18_V19_MC_UncertaintySources_AK4PFchs_EC2_2018', 'Regrouped_Autumn18_V19_MC_UncertaintySources_AK4PFchs_FlavorQCD', 'Regrouped_Autumn18_V19_MC_UncertaintySources_AK4PFchs_HF', 'Regrouped_Autumn18_V19_MC_UncertaintySources_AK4PFchs_HF_2018', 'Regrouped_Autumn18_V19_MC_UncertaintySources_AK4PFchs_RelativeBal', 'Regrouped_Autumn18_V19_MC_UncertaintySources_AK4PFchs_RelativeSample_2018', 'Regrouped_Autumn18_V19_MC_UncertaintySources_AK4PFchs_Total', 'Summer19UL18_JRV2_MC_PtResolution_AK4PFchs', 

In [63]:
events.HLT.IsoMu24

[None, None, [70.88551330566406, 33.272789001464844]]

In [64]:
clean_jets[134:137].pt.to_list()

[None, None, [69.25, 32.9375]]

In [65]:
clean_jets[130:140].pt.to_list()

[None, None, None, None, None, None, [69.25, 32.9375], None, None, None]

In [68]:
def get_btagsf(eta, pt, flavor, year='2018', sys='nominal'):
  # Efficiencies and SFs for UL only available for 2016APV, 2017 and 2018
  # light flavor SFs and unc. missed for 2016APV
  if   (year == '2016' or year == '2016APV'): SFevaluatorBtag = BTagScaleFactor("data/btagSF/DeepFlav_2016.csv","MEDIUM") 
  elif year == '2017': SFevaluatorBtag = BTagScaleFactor("data/btagSF/UL/DeepJet_UL17.csv","MEDIUM")
  elif year == '2018': SFevaluatorBtag = BTagScaleFactor("/data/pubfs/xiaoj/test/DeepFlav_2018.csv","MEDIUM")
  else: raise Exception(f"Error: Unknown year \"{year}\".")

  return SFevaluatorBtag.eval("central",flavor,eta,pt), SFevaluatorBtag.eval("up",flavor,eta,pt), SFevaluatorBtag.eval("down",flavor,eta,pt)


In [69]:
isData=False
if not isData:
    pt = clean_jets.pt; abseta = np.abs(clean_jets.eta); flav = clean_jets.hadronFlavour

    clean_jets['btagSF'], clean_jets['btagSF_up'], clean_jets['btagSF_down']  = get_btagsf(abseta, pt, flav, '2018')


In [70]:
clean_jets.btagSF_down[134:137]

<Array [None, None, [1.27, 1.3]] type='3 * option[var * float64]'>

In [31]:
np.__version__

'1.22.0'

In [ ]:
ak.to_list(jtmp.pt)

[[57.96875, 19.421875], [47.28125, 35.3125, 28.03125], [35.96875, 15.609375]]

In [ ]:
ak.to_list(mtmp.pt)

[[53.41250228881836],
 [42.005924224853516, 31.437562942504883],
 [32.56406784057617]]

In [60]:
jtmp = ak.zip({
            "pt": jets.pt,
            "eta": jets.eta,
            "phi": jets.phi,
            "mass": jets.mass,
            "charge": np.ones(len(jets.pt)),
        }, with_name="PtEtaPhiMCandidate")
mtmp = ak.zip({
            "pt": good_muons.pt,
            "eta": good_muons.eta,
            "phi": good_muons.phi,
            "mass": good_muons.mass,
            "charge": good_muons.charge,
        }, with_name="PtEtaPhiMCandidate")
jtmp = jtmp[0:3]
mtmp = mtmp[0:3]

pair_obj = ak.cartesian([jtmp, mtmp],nested=True)
obj1, obj2 = ak.unzip(pair_obj)
obj1.delta_r(obj2)

UFuncTypeError: ufunc '_deltaphi_kernel' did not contain a loop with signature matching types (<class 'numpy.dtype[float32]'>, <class 'numpy.dtype[float32]'>) -> None

## Don't remove, the place to make the "clean" with cartesian clear

In [ ]:
jtmp = jets[0:3]
mtmp = good_muons[0:3]

pair_obj = ak.cartesian([jtmp, mtmp],nested=True)
obj1, obj2 = ak.unzip(pair_obj)
obj1.delta_r(obj2)

<Array [[[0.0113], [4.6, ... 0.00839], [1.43]]] type='3 * var * var * float32'>

In [ ]:
obj1.delta_r(obj2).to_list()

[[[0.011299419216811657], [4.601864337921143]],
 [[0.005097805988043547, 3.1803181171417236],
  [3.207820415496826, 0.028553511947393417],
  [4.776998043060303, 3.171268939971924]],
 [[0.00838739424943924], [1.430845856666565]]]

In [ ]:
obj1.pt.to_list()

[[[57.96875], [19.421875]],
 [[47.28125, 47.28125], [35.3125, 35.3125], [28.03125, 28.03125]],
 [[35.96875], [15.609375]]]

In [ ]:
obj2.pt.to_list()

[[[53.41250228881836], [53.41250228881836]],
 [[42.005924224853516, 31.437562942504883],
  [42.005924224853516, 31.437562942504883],
  [42.005924224853516, 31.437562942504883]],
 [[32.56406784057617], [32.56406784057617]]]

In [ ]:
a = obj1.delta_r(obj2)
ak.min(a,axis=2).to_list()

[[0.011299419216811657, 4.601864337921143],
 [0.005097805988043547, 0.028553511947393417, 3.171268939971924],
 [0.00838739424943924, 1.430845856666565]]

In [ ]:
jtmp.pt.to_list()

[[57.96875, 19.421875], [47.28125, 35.3125, 28.03125], [35.96875, 15.609375]]

In [ ]:
mtmp.pt.to_list()

[[53.41250228881836],
 [42.005924224853516, 31.437562942504883],
 [32.56406784057617]]

In [ ]:
jtmp[:,0].delta_r(mtmp).to_list()

[[0.011299419216811657],
 [0.005097805988043547, 3.1803181171417236],
 [0.00838739424943924]]

In [ ]:
jtmp[:,1].delta_r(mtmp).to_list()

[[4.601864337921143],
 [3.207820415496826, 0.028553511947393417],
 [1.430845856666565]]

In [ ]:
jtmp[:,0].delta_r(mtmp[:,0]).to_list()

[0.011299419216811657, 0.005097805988043547, 0.00838739424943924]

In [ ]:
a = obj1.delta_r(obj2)
b = ak.min(a,axis=2)
c = (b > 0.4)
clean_jets = jtmp[c]
(clean_jets.pt).to_list()

[[19.421875], [28.03125], [15.609375]]